In [1]:
# import os           #파일접근
# import glob         #파일 나열을위한
# import numpy as np              # 다차원 배열, 수학함수
# import matplotlib.pyplot as plt   #데이터 시각화 = 그래프 패키지
# import pandas as pd     #데이터를 저장하고 처리하는 패키지
# import json         # json
# import base64 
# %matplotlib inline

# import tensorflow as tf
# from tensorflow.keras import datasets
# from tensorflow.keras import layers
# from tensorflow.keras import models

# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
# from keras.applications.vgg16 import preprocess_input
# from keras.applications.vgg16 import decode_predictions
# from keras.applications.vgg16 import VGG16


# from tensorflow.keras.models import load_model
# from PIL import Image  # 이미지 클래스 사용

# # 숫자 -> One-hot Vector 를 위한 라이브러리
# from keras.utils import np_utils


In [2]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.backend import set_session as K
import tensorflow as tf
from keras import regularizers
from keras import layers, models
from keras.applications import VGG16
from keras import Input
from keras.models import Model
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
import os
from glob import glob
from PIL import Image
import numpy as np

# # 숫자 -> One-hot Vector 를 위한 라이브러리
from keras.utils import np_utils


### 데이터 load

In [3]:
labels = [ '트래디셔널', '매니시', '페미닌', '에스닉','컴템포러리',
          '내추럴', '젠더플루이드', '스포티', '서브컬쳐','캐주얼']

#####  전처리

In [4]:
# 이미지 데이터 준비
width=224
height=224
x_train=[]
x_test=[]
y_train=[]
y_test=[]
cnt=0
path1='./K-Fashion/Training/원천데이터-기준10/'
for style in os.listdir(path1):
    path2=path1+style
    cnt=0
    print(style)
    styleN = labels.index(style)
    print(styleN)
    for img in os.listdir(path2):
        cnt=cnt+1
        path3=path2+'/'+img
        image=Image.open(path3)
        image = image.resize((width,height), resample=0) # 사이즈 조정 
        image_array = np.array(image) 
        if cnt <= 3000 :
            x_train.append(image_array)
            y_train.append(styleN)
        elif 3000 < cnt and cnt <= 3500:
            x_test.append(image_array)
            y_test.append(styleN)


내추럴
5
매니시
1
서브컬쳐
8
스포티
7
에스닉
3
젠더플루이드
6
캐주얼
9
컴템포러리
4
트래디셔널
0
페미닌
2


In [5]:
# # 정답
# y_train=[]
# y_test=[]
# cnt=0
# path1='./K-Fashion/Training/라벨링데이터1/'
# for style in os.listdir(path1):
#     path2=path1+style
#     cnt=0
#     print(style)
#     for json in os.listdir(path2):
#         cnt=cnt+1
#         path3=path2+'/'+json
        
#         if cnt < 2 :
#             print(json)
# #             y_train.append(image_array)
# #         elif 500 <= cnt and cnt < 1000:
# #             y_test.append(image_array)



In [6]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)
y_train.shape

(26000,)

In [7]:
# Loss의 스케일 조정을 위해 0 ~ 255 -> 0 ~ 1 범위로 만들어줍니다
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# One-Hot Encoding (숫자 -> One-Hot Vector) 
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
# y의 최대값이 나오게된다
# [손흔드는 라이언] [오후 4:08] 아 현재 y의 값(라벨)이 3이라서
# [손흔드는 라이언] [오후 4:09] 0,1,2,3 이렇게 4개로 분류한다고
# [손흔드는 라이언] [오후 4:09] 되는걸까요?

In [8]:
x_train.shape, y_train.shape

((26000, 224, 224, 3), (26000, 10))

####  d

### 모델

In [9]:
# channel = 3

# # 모델 설정
# # 순차적 모델을 만들기 위한 Sequential() 함수
# model = models.Sequential()
# # filters : Kernel 그룹의 갯수 / kernel_size : Kernel 크기
# model.add(layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', 
#                  input_shape=(width, height, 3)))
# model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
# # model.add(layers.Dropout(0.2))

# model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
# model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
# # model.add(layers.Dropout(0.2))
# model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
# model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
# # N차원 배열을 1차원으로 펴줍니다
# model.add(layers.Flatten())
# # model.add(layers.Dense(64, activation='relu'))
# # 출력층 MLP  # y 종류수 
# model.add(layers.Dense(10,activation='softmax'))

# model.summary()

In [10]:
input_tensor = Input(shape=(width, height, 3), dtype='float32', name='input')

#vgg16 모델 불러오기
pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(width, height, 3))
pre_trained_vgg.trainable = False
pre_trained_vgg.summary()

#vgg16 밑에 레이어 추가
additional_model = models.Sequential()
additional_model.add(pre_trained_vgg)
additional_model.add(layers.Flatten())
additional_model.add(layers.Dense(4096, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
additional_model.add(layers.Dropout(0.5))
additional_model.add(layers.Dense(2048, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
additional_model.add(layers.Dropout(0.5))
additional_model.add(layers.Dense(1024, kernel_regularizer = regularizers.l1_l2
                                  (l1=0.001,l2=0.001),activation='relu'))
additional_model.add(layers.Dropout(0.5))
additional_model.add(layers.Dense(10, activation='softmax')) # 카테고리수

additional_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
# model.compile(loss='categorical_crossentropy', 
#               optimizer='sgd', 
#               metrics=['accuracy'])

In [12]:
# 100개 에폭 10, 배치 16  5시 45분 

In [13]:
# hist = model.fit(x_train, y_train,
#                  epochs=10,
#                  batch_size=16,
#                  validation_data=(x_test, y_test),
#                 verbose=1,
#                 shuffle=True)

In [ ]:
history = additional_model.fit(x_train, y_train, 
                    batch_size=16, 
                    epochs=10, 
                    validation_data=(x_test, y_test))

Epoch 1/10
1625/1625 [==============================] - 5746s 4s/step - loss: 38.0451 - acc: 0.1134 - val_loss: 6.0181 - val_acc: 0.0000e+00
Epoch 2/10
1335/1625 [=======================>......] - ETA: 17:53 - loss: 5.9934 - acc: 0.1139

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()